In [ ]:
import os
import re
from pathlib import Path
from medpy.metric import assd, dc, precision, recall
import SimpleITK as sitk
import numpy as np
import csv
from tqdm import tqdm_notebook as tqdm

In [ ]:
labels = ['Cartilage', 'Labrum']
folds_path = '/mnt/Data/nnUNet_trained_models/nnUNet/3d_fullres/Task604_Cartilage_Labrum_TRUFI/nnUNetTrainerV2__nnUNetPlansv2.1/'
gts_path = '/mnt/Data/nnUNet_trained_models/nnUNet/3d_fullres/Task604_Cartilage_Labrum_TRUFI/nnUNetTrainerV2__nnUNetPlansv2.1/gt_niftis/'
folds_path = Path(folds_path)
gts_path = Path(gts_path)
pred = [str(file) for file in folds_path.glob('*/validation_raw_postprocessed/*.nii.gz')]
gts = [str(file) for file in gts_path.glob('*.nii.gz')]

data = []
for case in pred:
    id_ = re.search(r'\d{3}(?=.nii.gz)', case).group(0)
    print(id_)
    data.append({'ID':int(id_),
                 'Set': 'TRUFI_5Fold',
                  'pred': case,
                  'gt': [path for path in gts if id_+'.nii' in path][0]
        })
    for label in labels:
        data[-1][label] = {}

In [ ]:
folds_path = '/mnt/Data/nnUNet_trained_models/nnUNet/3d_fullres/Task600_Cartilage_Labrum/nnUNetTrainerV2__nnUNetPlansv2.1/'
gts_path = '/mnt/Data/nnUNet_trained_models/nnUNet/3d_fullres/Task600_Cartilage_Labrum/nnUNetTrainerV2__nnUNetPlansv2.1/gt_niftis/'
folds_path = Path(folds_path)
gts_path = Path(gts_path)
pred = [str(file) for file in folds_path.glob('*/validation_raw_postprocessed/*.nii.gz')]
gts = [str(file) for file in gts_path.glob('*.nii.gz')]

for case in pred:
    id_ = re.search(r'\d{3}(?=.nii.gz)', case).group(0)
    print(id_)
    data.append({'ID':int(id_),
                 'Set': 'MP2RAGE_5Fold',
                  'pred': case,
                  'gt': [path for path in gts if id_+'.nii' in path][0]
        })
    for label in labels:
        data[-1][label] = {}

In [ ]:
pred_path = '/mnt/Data/nnUNet_raw/nnUNet_raw_data/Task600_Cartilage_Labrum/imagesTsPred'
gts_path = '/mnt/Data/nnUNet_raw/nnUNet_raw_data/Task600_Cartilage_Labrum/labelsTs'
pred_path = Path(pred_path)
gts_path = Path(gts_path)
pred = [str(file) for file in pred_path.glob('*.nii.gz')]
gts = [str(file) for file in gts_path.glob('*.nii.gz')]

for case in pred:
    id_ = re.search(r'\d{3}(?=.nii.gz)', case).group(0)
    print(id_)
    data.append({'ID':int(id_),
                 'Set': 'MP2RAGE_Test',
                  'pred': case,
                  'gt': [path for path in gts if id_+'.nii' in path][0]
        })
    for label in labels:
        data[-1][label] = {}

In [ ]:
pred_path = 'External TRUFI/External_validation_TRUFI_Cropped/labelsTsPredict'
gts_path = 'External TRUFI/External_validation_TRUFI_Cropped/labelsTs'
pred_path = Path(pred_path)
gts_path = Path(gts_path)
pred = [str(file) for file in pred_path.glob('*[0-9].nii.gz')]
gts = [str(file) for file in gts_path.glob('*.nii.gz')]

for case in pred:
    id_ = re.search(r'\d{3}(?=.nii.gz)', case).group(0)
    print(id_)
    data.append({'ID':int(id_),
                 'Set': 'MP2RAGE_TRUFI_Test',
                  'pred': case,
                  'gt': [path for path in gts if id_+'.nii' in path][0]
        })
    for label in labels:
        data[-1][label] = {}


In [ ]:
pred_path = 'External TRUFI/second_set/labelsTsPredict'
gts_path = 'External TRUFI/second_set/labelsTs'
pred_path = Path(pred_path)
gts_path = Path(gts_path)
pred = [str(file) for file in pred_path.glob('*[0-9].nii.gz')]
gts = [str(file) for file in gts_path.glob('*.nii.gz')]

for case in pred:
    id_ = re.search(r'\d{3}(?=.nii.gz)', case).group(0)
    print(id_)
    data.append({'ID':int(id_),
                 'Set': 'TRUFI_Test',
                  'pred': case,
                  'gt': [path for path in gts if id_+'.nii' in path][0]
        })
    for label in labels:
        data[-1][label] = {}

In [ ]:
len(data)

In [ ]:
lssGTFilter = sitk.LabelShapeStatisticsImageFilter()
lssPredFilter = sitk.LabelShapeStatisticsImageFilter()
for patient in tqdm(data):
    gt = sitk.ReadImage(patient['gt'])
    pred = sitk.ReadImage(patient['pred'])
    spacing = np.flip(np.array(gt.GetSpacing()))
    gt_np = sitk.GetArrayFromImage(gt)
    pred_np = sitk.GetArrayFromImage(pred)
    
    lssGTFilter.Execute(gt)
    lssPredFilter.Execute(pred)
    
    
    for num, label in enumerate(labels, start = 1):
        gt_label = gt_np == num
        pred_label = pred_np == num
        
        patient[label]['DC'] = dc(pred_label,gt_label)
        patient[label]['ASSD'] = assd(pred_label,gt_label,voxelspacing=spacing)
        patient[label]['Precision'] = precision(pred_label,gt_label)
        patient[label]['Recall'] = recall(pred_label,gt_label)
        patient[label]['manual_Volume'.format(label)] = lssGTFilter.GetPhysicalSize(num)
        patient[label]['automatic_Volume'.format(label)] = lssPredFilter.GetPhysicalSize(num)


In [ ]:
for label in labels:
    print(label)
    for key in patient[label].keys():
        print(key, np.mean([patient[label][key] for patient in data]))

In [ ]:
[(patient['Cartilage']['manual_Volume'],patient['Cartilage']['automatic_Volume']) for patient in data]

In [ ]:
with open('paper_stats.csv', 'w', newline='') as csvfile:
    fieldnames = ['ID', 'Set','Label','DC', 'ASSD', 'Precision','Recall','manual_Volume', 'automatic_Volume']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for patient in data:
        for label in labels:
            writer.writerow({'ID':patient['ID'],
                             'Set':patient['Set'],
                             'Label': label,
                             'DC':patient[label]['DC'],
                             'ASSD':patient[label]['ASSD'],
                             'Precision':patient[label]['Precision'],
                             'Recall':patient[label]['Recall'],
                             'manual_Volume':patient[label]['manual_Volume'],
                             'automatic_Volume':patient[label]['automatic_Volume']
                            })